In [5]:
import geemap
import ee
import matplotlib
import pandas as pd
import pytz


In [2]:
ee.Initialize()

In [3]:
N = 45.40682960240968 
E = 4.747075353314043
# turn into a gee point
point = ee.Geometry.Point([E, N])
start_date = '2024-06-01'
end_date = '2024-06-30'

In [4]:
# check that
Map = geemap.Map()
Map.addLayer(point)
Map.centerObject(point)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# Get SAT dates/times

In [7]:
# get each collection 
ASTER = ee.ImageCollection("ASTER/AST_L1T_003").filterDate(start_date, end_date).filterBounds(point)
LS9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate(start_date, end_date).filterBounds(point)
LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date, end_date).filterBounds(point)

# LS8.size().getInfo()

In [13]:
# get the dates and times
def printOutDates(ic, year=False, op=False, riverName=False):
    '''
    extracts a df of dates and times from an image collection in GEE
    Should already have cloud etc accounted for 

    Parameters
    ----------
    ic : GEE image collection
        this is built to be within the LandsatGEE functions.
        
    A saving function has been left in but will not be used in the main processing algorithm for the paper
    year : str, optional
        The default is False.
    op : str, optional
        The default is False.
    riverName : str, optional
         The default is False.

    Returns
    -------
    df : TYPE
        DESCRIPTION.

    '''
    dates = []
    times = []
    try:
        icList = ic.toList(ic.size())
        icInfo = icList.getInfo()

    except:
        # print('Ic empty')
        # print(ic.size().getInfo())
        # returning an empty dataframe enables generate dates to run and just not include this satellite
        return pd.DataFrame({})
    tz = pytz.timezone('Europe/Paris')

    for im in icInfo:
        
        dt = pd.to_datetime(im['properties']['system:time_start'], unit='ms')
#         dt = dt.tz_localize('UTC').tz_convert(tz)

        date = dt.date()
        time = dt.time()
        
        dates.append(date)
        times.append(time)
    df = pd.DataFrame({'date':dates, 'time':times})
    
    ## effectively removed due to processing functioning running without - to save duplicate saving of dates
    if op != False:
        # check path for river exists
        op = os.path.join(op, (riverName+str(year)+'.csv'))
        df.to_csv(op)
        
    return df


In [14]:
# print these 
A_df = printOutDates(ASTER)
ls9_df = printOutDates(LS9)
ls8_df= printOutDates(LS8)


In [15]:
# ASTER
dfs = [A_df, ls9_df, ls8_df]
names = ['Aster', 'Landsat9', 'Landsat8']
for a, name in zip(dfs, names):
    a['sat'] = name
    print(a)

         date      time    sat
0  2024-06-16  10:00:01  Aster
         date             time       sat
0  2024-06-02  10:22:27.432000  Landsat9
1  2024-06-18  10:22:14.319000  Landsat9
2  2024-06-02  10:22:51.315000  Landsat9
3  2024-06-18  10:22:38.197000  Landsat9
4  2024-06-25  10:28:18.883000  Landsat9
         date             time       sat
0  2024-06-10  10:22:10.101000  Landsat8
1  2024-06-26  10:22:21.797000  Landsat8
2  2024-06-10  10:22:33.988000  Landsat8
3  2024-06-26  10:22:45.684000  Landsat8
4  2024-06-17  10:28:25.844000  Landsat8


In [10]:
# save
final = pd.concat(dfs).sort_values('date')

In [11]:
op = r'C:\Users\lgxsv2\OneDrive - The University of Nottingham\PhD\yr_2\18_PaperThree\RT_temperaturePrivate\Analysis\Figures\laptopBased_overpassTime\GEEDates.csv'

In [12]:
final.to_csv(op)